EN la pregunta se utilizo la funncion para adaptar los datos con csv pasando por parametro el dataset original, las columnas a modificar y adaptar, las columnas a eliminar se elimino la columna 0 que era fecha no lo vi conveniente para la prediccion despues otra funcion  para aumentar mas datos sinteticos, hasta llegar a 10000, de los cuales se contaban 1462, despues se procede con las demas funciones y sacamos el valor predicho. este cuadernillo es de clasificacion multiclase y use one vs all para poder clasificar y la regresion a usar es conjugation gradient con un lambda de 0.3
nombre: SAMUEL DAYLER AMONZABEL GONZALES
CARRERA: INGNEIERAI DE SISTEMAS PREGUNTA # 1

In [196]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [197]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [198]:
data = pd.read_csv('/content/drive/MyDrive/DataSets/seattle-weather.csv')
print(data)

            date  precipitation  temp_max  temp_min  wind  weather
0     2012-01-01            0.0      12.8       5.0   4.7  drizzle
1     2012-01-02           10.9      10.6       2.8   4.5     rain
2     2012-01-03            0.8      11.7       7.2   2.3     rain
3     2012-01-04           20.3      12.2       5.6   4.7     rain
4     2012-01-05            1.3       8.9       2.8   6.1     rain
...          ...            ...       ...       ...   ...      ...
1456  2015-12-27            8.6       4.4       1.7   2.9     rain
1457  2015-12-28            1.5       5.0       1.7   1.3     rain
1458  2015-12-29            0.0       7.2       0.6   2.6      fog
1459  2015-12-30            0.0       5.6      -1.0   3.4      sun
1460  2015-12-31            0.0       5.6      -2.1   3.5      sun

[1461 rows x 6 columns]


In [199]:
def csvAdapter(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')
    columna = data.columns #asignamos los nombres de columna a la variables columna

    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())#obetenemos todos lo valores unicos de cada columna
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))])))  
    data = data.drop([columna[index]for index in eliminarColumnas],axis=1)#eliminamos las columnas   
    data = data.dropna()#eliminamos las filas que contengan valores nulos
    #data = pd.DataFrame(data, columns=columna)
    #dataN = data.to_numpy()
    print(data.head())
    return data
rutadataset='/content/drive/MyDrive/DataSets/seattle-weather.csv'
columnaCategoricas = [i for i in range(0, 6)]
eliminarColumnas=[0]
data = csvAdapter(rutadataset, columnaCategoricas, eliminarColumnas)

   precipitation  temp_max  temp_min  wind  weather
0              0         0         0     0        0
1              1         1         1     1        1
2              2         2         2     2        1
3              3         3         3     0        1
4              4         4         1     3        1


In [200]:
def generarDatosSinteticos(data):
    # La cantidad de filas a generar los datos sisteticos son los faltantes a 10000
    n_filas = 10000 - len(data)
    nuevos_datos = np.empty([n_filas, len(data.columns)])
    for i in range(len(data.columns)):
        # Obtener la columna i del DataFrame de entrada
        columna_i = data.iloc[:, i]
        # Generar muestras aleatorias con reemplazo de la columna i
        muestras_i = np.random.choice(columna_i, n_filas, replace=True)
        # Asignar las muestras generadas a la columna i del arreglo de nuevos datos
        nuevos_datos[:, i] = muestras_i
    # Crear un DataFrame con los nuevos datos sintéticos
    columnas = list(data.columns)
    nuevos_datos_df = pd.DataFrame(nuevos_datos, columns=columnas)
    # Concatenar los datos existentes con los nuevos datos sintéticos
    nuevoDatos = pd.concat([data, nuevos_datos_df], axis=0)
    # Devolver el DataFrame extendido como un arreglo NumPy
    return nuevoDatos

data = generarDatosSinteticos(data)
print(len(data))

10000


In [201]:
num_labels = 5
data = data.to_numpy()
X, y = data[:, 0:4], data[:, 4]
m = y.size

In [202]:
print(X)
print(y)

[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 2.  2.  2.  2.]
 ...
 [ 0. 27. 15. 29.]
 [ 0. 33.  0. 41.]
 [ 0.  4. 27.  8.]]
[0. 1. 1. ... 4. 2. 1.]


In [203]:
def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Muestra datos 2D almacenados en X en una cuadrícula apropiada.
    """
    # Calcula filas, columnas
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promocionar a una matriz bidimensional
    else:
        raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    # Calcula el numero de elementos a mostrar
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

In [204]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [205]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [206]:
h = sigmoid(0)
print(h)

0.5


In [207]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y 
    el gradiente del costo w.r.t. a los parámetros.
    
    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas 
        incluida la intercepcion
        
    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de 
        caracteristicas (incluida la intercepcion).
    
    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos
    
    lambda_ : float
        Parametro de regularización. 
    
    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada. 
    
    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la 
        función de costo con respecto a theta, en los valores actuales de theta..
    """
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [208]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [209]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 5000}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [210]:
lambda_ = 0.3
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)
all_theta

(5, 5)


array([[-3.17212840e+00, -8.85156420e-03,  1.61027998e-03,
        -5.19258256e-03,  2.35229598e-03],
       [-9.91238360e-02,  7.99811675e-03, -5.19665918e-03,
        -6.27782157e-03,  6.02615686e-04],
       [-4.27360458e-01, -6.78814369e-03,  5.03115338e-03,
         7.71989335e-03, -7.83607090e-04],
       [-3.80117415e+00,  1.19710673e-03, -2.73942363e-03,
        -7.61208221e-03, -6.96616343e-04],
       [-2.59948478e+00, -2.03935215e-03,  4.40574071e-04,
        -6.42284363e-04, -4.52991278e-04]])

In [211]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [212]:
def clima(clave):
    clima = {
        1: 'Sol',
        2: 'Lluvia',
        3: 'Llovizna',
        4: 'Nieve',
        5: 'Niebla'
    }
    
    estado = clima.get(clave)
    
    if estado:
        return estado
    else:
        return 'Estado del clima desconocido'

In [216]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[7500:7501, :].copy()
print(XPrueba.shape)
print(XPrueba)
XPrueba = np.concatenate([np.ones((1, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)
print("La prueba es: ", clima(p[0]))




(10000, 4)
Precision del conjuto de entrenamiento: 47.48%
(1, 4)
[[ 0.  7. 43.  8.]]
(1, 5)
[2]
La prueba es:  Lluvia
